# Question 1: 5 pts
Download the adjective-noun data sets here.

Load it into spark and use divide-and-conquer to find the first (adj, noun) pair in which the noun is 'unification'. Print the corresponding adjective.  The skeleton code is provided below.  One solution is to use filter() to find all pairs where the noun is 'unification', and then report the first one.  This is inefficient.  The better idea is to find, in parallel, the first such pair in each partition (if one exists), and then find the first partition that returns such a pair.

In [92]:
numPartitions = 10
path_to_file = "adj_noun_pairs.txt"

lines = sc.textFile(path_to_file, numPartitions)
pairs = lines.map(lambda l: tuple(l.split())).filter(lambda p: len(p) == 2)
pairs.cache()

# FILL IN YOUR CODE HERE
def first_pair(iterator):
    for i in iterator:
        if i[1] == "unification":
            yield i
            break

print(pairs.mapPartitions(first_pair).collect()[0][0])

several


## Sample Answer

In [ ]:
def f(iterator):   
  for i in iterator:     
    if i[1] == 'unification':       
      yield i[0]
      break
print pairs.mapPartitions(f).take(1)

# Question 2: 5 pts
Design a parallel divide-and-conquer algorithm for the following problem: Given two strings of equal length, compare them lexicographically. Output '<', '=', or '>', depending on the comparison result. The skeleton code is provided below.  Your code should run on all partitions of the rdd in parallel.

In [93]:
x = 'abcccbcbcacaccacaabb'
y = 'abcccbcccacaccacaabb'

numPartitions = 4
rdd = sc.parallelize(zip(x,y), numPartitions)

# FILL IN YOUR CODE HERE
def compare(iterator):
    c = '='
    
    for x, y in iterator:
        if x < y:
            c = '<'
            break
        elif x > y:
            c = '>'
            break
            
    yield c
        
compared_partitions = rdd.mapPartitions(compare).collect()

In [94]:
compared_partitions

['=', '<', '=', '=']

In [95]:
try:
    print(next(item for item in compared_partitions if item is not '='))
except StopIteration:
    print('=')

<


## Sample Answer

In [ ]:
def f(iterator):
  for i in iterator:
    if (i[0] < i[1]):
      yield -1
      return
    elif (i[0] > i[1]):
      yield 1
      return
  yield 0

result = rdd.mapPartitions(f).collect()
equal = True
for i in result:
  if (i == 1):
    print '>'
  elif (i == -1):
    print '<'
  else:
    continue
  equal = False
  break

if equal:
  print '='